## MiniAutoGen: Implementing a Multi-Agent Chat with Advanced Pipelines

The following example showcases the power and flexibility of MiniAutoGen, illustrating how to set up a multi-agent chat environment complete with sophisticated pipelines integrating LLMs, user interactions, and custom logic.

We are using some pre build components, you can see this [here.](../miniautogen/pipeline/components/components.py)

**Prerequisites**

Ensure you have the MiniAutoGen library installed and an OpenAI API key available. This example also uses the dotenv library for environment management.

In [7]:
#Initializing LLM Clients

from miniautogen.llms.llm_client import LiteLLMClient
litellm_client = LiteLLMClient(model="ollama/mistral-openorca") #We he are using a local model on OLLAMA.


In [8]:
# Building the Chat Environment

from miniautogen.chat.chat import Chat
from miniautogen.agent.agent import Agent
from miniautogen.chat.chatadmin import ChatAdmin
from miniautogen.pipeline.pipeline import Pipeline
from miniautogen.pipeline.components.components import (
    UserResponseComponent, AgentReplyComponent, TerminateChatComponent,
    Jinja2SingleTemplateComponent, LLMResponseComponent, NextAgentSelectorComponent
)

# Define a Jinja2 template for formatting messages
template_str = """
[{"role": "system", "content": "{{ agent.role }}"}{% for message in messages %},
  {% if message.sender_id == agent.agent_id %}
    {"role": "assistant", "content": {{ message.message | tojson | safe }}}
  {% else %}
    {"role": "user", "content": {{ message.message | tojson | safe }}}
  {% endif %}
{% endfor %}]
"""

# Initialize Jinja2 component with the template
jinja_component = Jinja2SingleTemplateComponent()
jinja_component.set_template_str(template_str)

# Set up pipelines for different components
pipeline_user = Pipeline([UserResponseComponent()])
pipeline_jinja = Pipeline([jinja_component, LLMResponseComponent(litellm_client)])
pipeline_admin = Pipeline([NextAgentSelectorComponent(), AgentReplyComponent(), TerminateChatComponent()])

# Create the chat environment
chat = Chat()

# Define agents with JSON data
json_data = {'agent_id': 'Bruno', 'name': 'Bruno', 'role': 'user'}
agent1 = Agent.from_json(json_data)
agent1.pipeline = pipeline_user  # Assign the user pipeline to agent1

agent2 = Agent("dev", "Carlos", "Python Senior Developer")
agent2.pipeline = pipeline_jinja  # Assign the LLM pipeline to agent2

# Add agents to the chat
chat.add_agent(agent1)
chat.add_agent(agent2)

# Add test messages to the chat
json_messages = [{'sender_id': 'Bruno', 'message': 'It’s a test, don’t worry'}]
chat.add_messages(json_messages)

# Initialize and configure ChatAdmin
chat_admin = ChatAdmin("admin", "Admin", "admin_role", pipeline_admin, chat, 10)

In [9]:
#running the chat
chat_admin.run()

INFO:miniautogen.chat.chatadmin:Chat Admin started.
INFO:miniautogen.chat.chatadmin:Executing round 1


[{'role': 'system', 'content': 'Python Senior Developer'}, {'role': 'user', 'content': 'It’s a test, don’t worry'}]


INFO:miniautogen.chat.chatadmin:Executing round 2


 As a Python Senior Developer, I understand the importance of ensuring that your application performs at its best. To do this, I will take the following steps:

1. Review and analyze the current codebase to identify any potential bottlenecks or inefficiencies.
2. Identify areas where optimizations can be made, such as refactoring complex functions, reducing unnecessary calls or loops, or improving algorithmic efficiency.
3. Implement performance improvements using profiling tools and benchmarking techniques to ensure that the changes have a positive impact on overall performance.
4. Conduct thorough testing and integration with existing systems to ensure compatibility and maintainability of the optimized codebase.
5. Monitor and track any changes in performance over time to validate the effectiveness of the implemented improvements, as well as to identify any new potential bottlenecks or opportunities for further optimization.
6. Document and share best practices for optimizing Python 

INFO:miniautogen.chat.chatadmin:Executing round 3


Create a calc function on python.
[{'role': 'system', 'content': 'Python Senior Developer'}, {'role': 'user', 'content': 'It’s a test, don’t worry'}, {'role': 'assistant', 'content': ' As a Python Senior Developer, I understand the importance of ensuring that your application performs at its best. To do this, I will take the following steps:\n\n1. Review and analyze the current codebase to identify any potential bottlenecks or inefficiencies.\n2. Identify areas where optimizations can be made, such as refactoring complex functions, reducing unnecessary calls or loops, or improving algorithmic efficiency.\n3. Implement performance improvements using profiling tools and benchmarking techniques to ensure that the changes have a positive impact on overall performance.\n4. Conduct thorough testing and integration with existing systems to ensure compatibility and maintainability of the optimized codebase.\n5. Monitor and track any changes in performance over time to validate the effectivenes

INFO:miniautogen.chat.chatadmin:Executing round 4


 def calc(num1, num2, operation):
    if operation == '+':
        return num1 + num2
    elif operation == '-':
        return num1 - num2
    elif operation == '/':
        return num1 / num2
    elif operation == '*':
        return num1 * num2
    else:
        raise ValueError(f"Invalid operation {operation}")

Example usage:
num1 = 5
num2 = 3
operation = '+'
result = calc(num1, num2, operation)
print(result)


INFO:miniautogen.chat.chatadmin:Chat Admin stopped.
INFO:miniautogen.chat.chatadmin:Chat Admin stopped.


Terminate
Terminating chat...


In [11]:
chat.get_messages().to_dict(orient='records')

[{'id': 1,
  'sender_id': 'Bruno',
  'message': 'It’s a test, don’t worry',
  'timestamp': Timestamp('2024-01-03 15:06:02.518774'),
  'additional_info': None},
 {'id': 2,
  'sender_id': 'dev',
  'message': ' As a Python Senior Developer, I understand the importance of ensuring that your application performs at its best. To do this, I will take the following steps:\n\n1. Review and analyze the current codebase to identify any potential bottlenecks or inefficiencies.\n2. Identify areas where optimizations can be made, such as refactoring complex functions, reducing unnecessary calls or loops, or improving algorithmic efficiency.\n3. Implement performance improvements using profiling tools and benchmarking techniques to ensure that the changes have a positive impact on overall performance.\n4. Conduct thorough testing and integration with existing systems to ensure compatibility and maintainability of the optimized codebase.\n5. Monitor and track any changes in performance over time to va